# Algoritmos de ordenamiento

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
import sys

## Definición función `merge_sort`

In [ ]:
def merge(left: np.array, right: np.array) -> np.array:
    res = list()
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            res = np.append(res, left[0])
            left = np.delete(left, 0)
        else:
            res = np.append(res, right[0])
            right = np.delete(right, 0)
    if len(left) == 0:
        return np.append(res, right)
    else:
        return np.append(res, left)


def merge_sort(arr: np.array) -> np.array:
    size = len(arr)
    if size < 2:
        return arr
    left, right =  arr[:size // 2], arr[size // 2:]
    left = merge_sort(left)
    right = merge_sort(right)
    return merge(left, right)


## Definición función `quick_sort`

In [ ]:
def partition(arr: np.array, low: int, high: int) -> int:
    pivot = arr[high]
    i = low - 1
    for j in range(low, high):
        if arr[j] <= pivot:
            i += 1
            arr[i], arr[j] = arr[j], arr[i]
    arr[i + 1], arr[high] = arr[high], arr[i + 1]
    return i + 1

def quick_sort(arr: np.array, low: int, high: int):
    if low < high:
        p = partition(arr, low, high)
        quick_sort(arr, low, p - 1)
        quick_sort(arr, p + 1, high)

## Medir los tiempos de los distintos algoritmos

In [ ]:
sys.setrecursionlimit(1_000_000)

In [ ]:
LIMIT = 8
SEP = '\t\t'
problem_size = np.array([10 ** i for i in range(1, LIMIT + 1)])
ms_times = list()
qs_times = list()
np_times = list()

print(*'size,numpy quick sort,merge sort      ,quick sort'.split(','), sep= SEP)

for i in problem_size:
    array = np.random.randint(1, 100, i)


    print(i, end= SEP)
    start = time.time()
    np.sort(array)
    np_times.append(time.time() - start)
    print(np_times[-1], end= SEP)

    start = time.time()
    merge_sort(array)
    ms_times.append(time.time() - start)
    print(ms_times[-1], end= SEP)

    start = time.time()
    quick_sort(array, 0, len(array) - 1)
    qs_times.append(time.time() - start)
    print(qs_times[-1])


Guardar los tiempos en un data frame 

In [ ]:
times_df = pd.DataFrame({
    '#problems': problem_size,
    'merge': ms_times,
    'quick': qs_times,
    'numpy': np_times 
})
times_df

In [ ]:
sns.lineplot(
    data= times_df[['merge', 'quick', 'numpy']],
    # x= '#problems',
    # y= ['merge', 'quick', 'numpy'],
    legend= True
)
!clear